In [1]:
import pandas as pd
import numpy as np


In [2]:
df = pd.read_csv('D:/bmsAnomalyDetection/application/Anomaly-detection/notebook/data/labelled_data.csv')

In [4]:
df['BMS_max_cell_temp_id'] = df['BMS_max_cell_temp_id'].astype('object')
df['BMS_state'] = df['BMS_state'].astype('object')
df['BMS_min_cell_temp_id'] = df['BMS_min_cell_temp_id'].astype('object')
df['BMS_max_cell_voltage_id'] = df['BMS_max_cell_voltage_id'].astype('object')
df['BMS_min_cell_voltage_id'] = df['BMS_min_cell_voltage_id'].astype('object')
df['OBC_mux'] = df['OBC_mux'].astype('object')
df['OBC_port_status'] = df['OBC_port_status'].astype('object')
df['OBC_overvoltage_fault'] = df['OBC_overvoltage_fault'].astype('object')
df['OBC_overcurrent_fault'] = df['OBC_overcurrent_fault'].astype('object')
df['OBC_port_weld_fault'] = df['OBC_port_weld_fault'].astype('object')
df['Anomaly'] = df['Anomaly'].astype('object')

df = df.drop(['Anomaly_Score','Unnamed: 0','timestamps'], axis = 1)

In [5]:
x = df.drop('Anomaly', axis =1)
y = df['Anomaly'].astype('int')

In [6]:
char = []
num = []

for i in x.columns:
    if x[i].dtype == 'object':
        char.append(i)
    else:
        num.append(i)

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# Numeric transformations: Impute missing values and scale
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Categorical transformations: Impute missing and OneHotEncode
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent'))
])

In [8]:
from sklearn.compose import ColumnTransformer

# Use ColumnTransformer to apply transformations
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num),
        ('cat', categorical_transformer, char)
    ])


In [9]:
from sklearn.ensemble import RandomForestClassifier

# Create the final pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])


In [10]:
from imblearn.over_sampling import RandomOverSampler

# x = df.drop('Anomaly', axis =1)
# y = df['Anomaly'].astype('int')

sampler = RandomOverSampler(random_state=0,sampling_strategy='auto')
x_resampled, y_resampled = sampler.fit_resample(x,y)

In [11]:
from sklearn.model_selection import train_test_split

x_train, x_test,y_train, y_test = train_test_split(x_resampled, y_resampled, test_size=0.3, random_state=101)

In [12]:
# Fit the pipeline on the training data
pipeline.fit(x_train, (y_train))

# Make predictions on the test data
y_pred = pipeline.predict(x_test)

In [13]:
from sklearn.metrics import accuracy_score

print(f'Accuracy: {accuracy_score(y_test, y_pred)}')

Accuracy: 0.9985786802030456


In [17]:
import joblib

# Save the preprocessor
joblib.dump(preprocessor,'D:/bmsAnomalyDetection/application/Anomaly-detection/notebook/artifacts/Preprocessor.pkl')

# Save the model
joblib.dump(pipeline, 'D:/bmsAnomalyDetection/application/Anomaly-detection/notebook/artifacts/Model.pkl')

['D:/bmsAnomalyDetection/application/Anomaly-detection/notebook/artifacts/Model.pkl']